In [ ]:
!pip install -U openpyxl selenium beautifulsoup4 lxml requests

In [ ]:
'''
注意事項:
下載對應的 ChromeDriver (web driver) 到程式檔案同一個目錄下後解壓縮，下載前記得對應版本編號。
連結: https://chromedriver.chromium.org/downloads

參考網頁:
[1] 東吳中文學報
https://web-ch.scu.edu.tw/chinese/file/3423
[2] sqlite3 --- SQLite 数据库 DB-API 2.0 接口模块
https://docs.python.org/zh-tw/3/library/sqlite3.html
'''


'''
匯入套件
'''
# HTML parser
from bs4 import BeautifulSoup as bs

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 處理下拉式選單的工具
from selenium.webdriver.support.ui import Select

# 強制等待 (執行期間休息一下)
from time import sleep

# pretty-print
from pprint import pprint

# 隨機
from random import randint

# 計時
import time

# 整理 json 使用的工具
import json

# 執行 shell command 的時候用的
import os

# 取得錯誤訊息
import sys, traceback

# 子處理程序，用來取代 os.system 的功能
import subprocess

# 正規表達式
import re

# 編碼
from urllib.parse import quote

# SQLite 資料庫
import sqlite3

# 存取 Excel 的工具
from openpyxl import load_workbook
from openpyxl import Workbook

# 取得系統時間的工具
from datetime import datetime

# 引入 hashlib 模組
import hashlib

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")             #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")        #最大化視窗
my_options.add_argument("--incognito")              #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 給 web driver 用的變數
driver = None

# 來源首頁
prefix_url = 'https://web-ch.scu.edu.tw'
url = prefix_url + '/chinese/file/3423'

# 指定 sheet name
sheetName = 'web_ch_scu'

# 指定 excel 檔名
excelFileName = 'web_ch_scu.xlsx'

# 指定 json 檔名
jsonFileName = f'{sheetName}.json'

# 建立儲存檔案用的資料夾
folderPath = f'./{sheetName}'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

#預設下載路徑
my_options.add_experimental_option("prefs", {
    "download.default_directory": folderPath
})

# 放置爬取的資料
listData = []

# 分頁網址
listPage = []

In [ ]:
'''
函式
'''
# md5 (用來為每一筆資料建立唯一代號)
def md5(string):
    m = hashlib.md5()
    m.update(string.encode("utf-8"))
    return m.hexdigest()

# 初始化 Web Driver
def init():
    global driver
    # 使用 Chrome 的 WebDriver
    driver = webdriver.Chrome( 
        options = my_options, 
        executable_path = driver_exec_path
    )
    
# 取得每個分頁連結
def getPagination():
    global driver
    global listPage
    
    try:
        # 走訪首頁
        driver.get(url)
        
        # 因為進去即是第 1 頁，所以要先預設第 1 個連結
        setPage = set()
        setPage.add(f"{url}?page=1")
    
        # 等待目標元素出現
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'ul.pagination > li a')
            )
        )
        
        # 取得分頁列表
        a_elms = driver.find_elements(
            By.CSS_SELECTOR, 'ul.pagination > li a[href]'
        )
        
        # 開啟分頁
        for a in a_elms:
            # 取得不重複的分頁
            setPage.add(a.get_attribute('href'))
            
        # set 轉成 list
        listPage = list(setPage)
        listPage.sort()
        
        pprint(listPage)
            
    except TimeoutException as e:
        print('等待逾時: getPagination')
    
# 剖析內容
def parse():
    global driver, listData, listPage
    try:
        # 流水號
        sn = 1
        
        # 走訪每一分頁，同時為分頁底下的期刊連結開啟分頁
        for idx, link in enumerate(listPage):
            # 切換到初始分頁
            driver.switch_to.window(driver.window_handles[0])
            
            # 等待
            sleep(2)
            
            # 各自走訪首頁
            driver.get(link)
            print(link)
            
            # 等待目標元素出現
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (
                        By.CSS_SELECTOR, 
                        'table.table.table-striped.table-bordered tr[class]:not([class="info"])'
                    )
                )
            )
                
            # 開啟分頁，各別取得內頁資料
            a_elms = driver.find_elements(By.CSS_SELECTOR, 'table.table.table-striped.table-bordered tr[class]:not([class="info"]) td a')
            for index, a in enumerate(a_elms):
                # 開啟新分頁
                driver.execute_script(f'window.open("about:blank", "_blank");')
                
            # 將所有 tabs 轉址到分頁網址去，以便取得對應 a 列表
            for index, window in enumerate(driver.window_handles):
                if index == 0: continue
                driver.switch_to.window(driver.window_handles[index])
                driver.get(link)
                
            # 切換到初始分頁
            driver.switch_to.window(driver.window_handles[0])
                
            # 取得所有 tr
            tr_elms = driver.find_elements(By.CSS_SELECTOR, 'table.table.table-striped.table-bordered tr[class]:not([class="info"])')
            
            # 取得每一個 tr 底下的 td，並為 td 當中的 a 開啟分頁，之後在每一個分頁按下對應的 a，進入內頁
            for index, tr in enumerate(tr_elms):
                # 跳到指定分頁
                driver.switch_to.window(driver.window_handles[index + 1])
                
                # 等待
                driver.implicitly_wait(10)

                # 依連結順序按下連結
                a_elms = driver.find_elements(By.CSS_SELECTOR, 'table.table.table-striped.table-bordered tr[class]:not([class="info"]) td a')
                
                # 按下對應順序的連結，進入各自的內頁
                a_elms[index].click()
                
                print(a_elms[index].get_attribute('innerText'))

            # 切換到初始分頁
            driver.switch_to.window(driver.window_handles[0])
            
            # 取得所有 tr
            tr_elms = driver.find_elements(By.CSS_SELECTOR, 'table.table.table-striped.table-bordered tr[class]:not([class="info"])')
            
            # 取得所有 metadata，包括超連結
            for index, tr in enumerate(tr_elms):
                td_elms = tr.find_elements(By.CSS_SELECTOR, 'td')
                
                # 篇名
                strJournalTitle = td_elms[0].get_attribute('innerText')
                strJournalTitle = strJournalTitle.strip()
                
                # 出版日期
                strPublishDate = ''
                regexPublishDate = r'（(\d{4}\.\d{1,2})）|\((\d{4}\.\d{1,2}\.\d{1,2})\)'
                matchPublishDate = re.search(regexPublishDate, td_elms[1].get_attribute('innerText'))
                if matchPublishDate[1] != None:
                    strPublishDate = matchPublishDate[1]
                elif matchPublishDate[2] != None:
                    strPublishDate = matchPublishDate[2]
                    
                # 最後修訂日期
                strLastModifiedDate = td_elms[2].get_attribute('innerText')
                strLastModifiedDate = strLastModifiedDate.strip()

                # 跳到指定分頁
                driver.switch_to.window(driver.window_handles[index + 1])

                # 依連結順序按下連結
                strPdfLink = ''
                if len(driver.find_elements(By.CSS_SELECTOR, 'div#news_header ul li a[href]')) > 0:
                    a_elm = driver.find_element(By.CSS_SELECTOR, 'div#news_header ul li a[href]')
                    strPdfLink = a_elm.get_attribute('href')
                
                # 整理資料
                listData.append({
                    "id": md5(strJournalTitle),
                    "流水號": sn,
                    "篇名": strJournalTitle,
                    "出版日期": strPublishDate,
                    "最後修訂日期": strLastModifiedDate,
                    "檔案連結_原始": strPdfLink,
                    "檔案連結_curl": strPdfLink
                })
                
                # 遞增流水號
                sn += 1
                
                # 切換到初始分頁
                driver.switch_to.window(driver.window_handles[0])
                
            # 把所有分頁關掉
            while True:
                windows = driver.window_handles
                if len(windows) == 1: break
                driver.switch_to.window(windows[len(windows) - 1])
                driver.close()
                
            # 切換到初始分頁
            driver.switch_to.window(driver.window_handles[0])
            
    except TimeoutException as e:
        print('等待逾時: parse')
        
# 關閉瀏覽器
def close():
    driver.quit()
        
# 儲存成 json
def saveJson():
    global listData
    with open(f"{folderPath}/{jsonFileName}", "w", encoding="utf-8") as file:
        file.write( json.dumps( listData, ensure_ascii=False, indent=4 ) )
    
# 儲存 .db
def saveDB():
    with open(f"{folderPath}/{jsonFileName}", "r", encoding="utf-8") as file:      
        # 取得 json 內容
        strJson = file.read()
        
        # 將 json 轉成 list
        listJson = json.loads(strJson)
        
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{sheetName}.db")
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料
        sql_query = f'''
        SELECT id, title, link_curl, is_downloaded
        FROM journals
        WHERE id = ?
        '''
        
        # 寫入資料
        sql_insert = f'''
        INSERT INTO journals (
            id, title, publish_date, last_modified_date, link, 
            link_curl, is_downloaded, created_at, updated_at
        ) VALUES ( 
            ?,?,?,?,?,
            ?,?,?,?
        )
        '''
        
        for myDict in listJson:
            if cursor.execute(sql_query, (myDict["id"],)).fetchone() == None:
                cursor.execute(sql_insert, (
                    myDict['id'], 
                    myDict['篇名'],
                    myDict['出版日期'],
                    myDict['最後修訂日期'],
                    myDict['檔案連結_原始'],
                    myDict['檔案連結_curl'],
                    0,
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S"),
                    datetime.today().strftime("%Y-%m-%d %H-%M-%S")
                ))
        
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)

    # 關閉 sqlite
    conn.close()
    
# 下載
def download():
    # 寫入對話記錄
    conn = sqlite3.connect(f"{folderPath}/{sheetName}.db")
    conn.row_factory = sqlite3.Row
    
    # 建立 cursor 物件
    cursor = conn.cursor()

    # 執行 SQL 語法
    try:
        # 查詢特定資料
        sql_query = f'''
        SELECT sn, id, title, link_curl, is_downloaded
        FROM journals
        WHERE `is_downloaded` = 0
        '''
        
        # 更新資料的欄位(狀態)
        sql_update = f'''
        UPDATE `journals` 
        SET 
            `is_downloaded` = 1 ,
            `updated_at` = ?
        WHERE `id` = ?
        '''
        
        # 取得所有未下載的資料
        list_results = []
        for item in cursor.execute(sql_query).fetchall(): 
            list_results.append({k: item[k] for k in item.keys()})
        for myDict in list_results:
            # 等待
            sleep(randint(1,3))

            # 下載 pdf
            cmd = ['curl', '-L', myDict["link_curl"], '-o', f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf']
            result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            #output = result.stdout
            #pprint(output)
            print(f'{folderPath}/{myDict["sn"]}_{myDict["id"]}.pdf')

            # 將 is_downloaded 改成 1，代表已下載過
            cursor.execute(sql_update, (datetime.today().strftime("%Y-%m-%d %H-%M-%S"), myDict["id"],))
                
        conn.commit()
    except sqlite3.Error as err: 
        # 回滾
        conn.rollback()

        # SQLite3 例外處理
        exc_type, exc_value, exc_tb = sys.exc_info()
        strErrorMsg = f'''SQLite error: {' '.join(err.args)}\n\n
        SQLite traceback: {traceback.format_exception(exc_type, exc_value, exc_tb)}
        '''
        print(strErrorMsg)

    # 關閉 sqlite
    conn.close()

In [ ]:
# 初始化 Web Driver
init()

In [ ]:
# 取得每個分頁連結
getPagination()

In [ ]:
# 剖析內容
parse()

In [ ]:
# 關閉瀏覽器
close()

In [ ]:
# 儲存成 json
saveJson()

In [ ]:
# 儲存 .db
saveDB()

In [ ]:
# 下載
download()